In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv(r"C:\Users\Prarthana Saikia\Desktop\Praxis\Machine Learning\ML class\codes for exam\datasets\AnalyticsVidhyachallenge\train_HK6lq50.csv")
data.head()

,id,program_id,program_type,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y_1,Y,136,150,offline,intermediate,9389,M,Matriculation,3,24.0,5,N,1.0,0
1,16523_44,T_1,T,131,44,offline,easy,16523,F,High School Diploma,4,26.0,2,N,3.0,1
2,13987_178,Z_2,Z,120,178,online,easy,13987,M,Matriculation,1,40.0,1,N,2.0,1
3,13158_32,T_2,T,117,32,offline,easy,13158,F,Matriculation,3,NaN,4,N,1.0,1
4,10591_84,V_3,V,131,84,offline,intermediate,10591,F,High School Diploma,1,42.0,2,N,4.0,1


In [3]:
data.dtypes

id                            object
program_id                    object
program_type                  object
program_duration               int64
test_id                        int64
test_type                     object
difficulty_level              object
trainee_id                     int64
gender                        object
education                     object
city_tier                      int64
age                          float64
total_programs_enrolled        int64
is_handicapped                object
trainee_engagement_rating    float64
is_pass                        int64
dtype: object

In [4]:
data['is_pass'].value_counts()

1    50867
0    22280
Name: is_pass, dtype: int64

In [5]:
#replace missing values in age with mean and trainee engagement rating with mode(rating is 1.0)
r=data['age'].mean(axis=0)
data['age'].fillna(r,axis=0,inplace=True)
m=data['trainee_engagement_rating'].mode()
data['trainee_engagement_rating'].fillna(1.0,axis=0,inplace=True)


In [6]:
data_cat=data.iloc[:,[2,4,5,6,7,8,9,10,12,13,14]]
y=data.iloc[:,-1]

In [7]:
data_cat.head()

,program_type,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,total_programs_enrolled,is_handicapped,trainee_engagement_rating
0,Y,150,offline,intermediate,9389,M,Matriculation,3,5,N,1.0
1,T,44,offline,easy,16523,F,High School Diploma,4,2,N,3.0
2,Z,178,online,easy,13987,M,Matriculation,1,1,N,2.0
3,T,32,offline,easy,13158,F,Matriculation,3,4,N,1.0
4,V,84,offline,intermediate,10591,F,High School Diploma,1,2,N,4.0


In [8]:

from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
data_cat = data_cat.apply(encode.fit_transform)

In [9]:
data_cat.head()

,program_type,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,total_programs_enrolled,is_handicapped,trainee_engagement_rating
0,5,150,0,2,8688,1,3,2,4,0,0
1,1,44,0,0,15244,0,1,3,1,0,2
2,6,178,1,0,12916,1,3,0,0,0,1
3,1,32,0,0,12165,0,3,2,3,0,0
4,3,84,0,2,9808,0,1,0,1,0,3


In [10]:
#chi squared for feature selection

#performing chi2 test for feature selection 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
sel = SelectKBest(chi2,11)#this number 12 is a rsult after some hit and trials
a=sel.fit(data_cat, y)
score = (sel.scores_)
score

array([  4.97489879e+02,   1.17240013e+04,   1.00002382e+03,
         1.06625786e+03,   2.87359610e+05,   1.41590397e+01,
         3.90529034e+02,   2.83026665e+02,   7.14295695e+01,
         9.09831308e+01,   3.59892393e+03])

In [11]:
data_cat = sel.transform(data_cat)

In [12]:
data_cat.shape

(73147, 11)

In [13]:
data_num=data.iloc[:,[3,11]].values

In [14]:
X = np.concatenate((data_cat,data_num), axis = 1)

In [15]:
#Data Preprocessing
from sklearn.cross_validation import train_test_split
#X and y are arrays
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size = 0.2, random_state = 0)
y_train.value_counts()

C:\Users\Prarthana Saikia\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


1    40649
0    17868
Name: is_pass, dtype: int64

In [16]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(kind = 'regular')
x_resampled, y_resampled = sm.fit_sample(X_train, y_train)
print (np.bincount(y_resampled))



[40649 40649]


In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=600)

In [28]:
rf.fit(x_resampled, y_resampled)
y_pred1 = rf.predict(X_test)
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
roc_auc_score(y_test,y_pred1)

0.67899784693677823

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
gbtree = xgb.XGBClassifier(learning_rate=0.2,n_estimators=140, max_depth=10,min_child_weight=6)
#gbtree = xgb.XGBClassifier(subsample=0.8, min_child_weight= 10, max_depth= 4, gamma= 0.5, colsample_bytree= 1.0)
gbtree.fit(X_resampled, y_resampled)
y_pred5 = gbtree.predict(X_test)
roc_auc_score(y_test,y_pred5)

In [19]:
from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier     #baseclassifier


In [20]:
tree_class=DecisionTreeClassifier(max_depth=None)


In [22]:
tree_class.fit(x_resampled, y_resampled)
y_pred1 = tree_class.predict(X_test)
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
roc_auc_score(y_test,y_pred1)

0.64359421545928852

In [29]:
bag_tree=BaggingClassifier(base_estimator=tree_class,n_estimators=600,n_jobs=-1)
bag_tree.fit(x_resampled, y_resampled)
y_pred22 = bag_tree.predict(X_test)
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
roc_auc_score(y_test,y_pred22)

0.68219714130415687

In [26]:
#parameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50)
param_grid = { 
    'n_estimators': [100, 200,300,400,500,600,700],
    'max_features': ['auto', 'sqrt', 'log2']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_resampled, y_resampled)
print (CV_rfc.best_params_)

{'max_features': 'sqrt', 'n_estimators': 600}
